In [ ]:
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange # conda install -c conda-forge tqdm
%matplotlib notebook

rng = np.random.default_rng()

In [ ]:
def plotIsing(lattice):
    up   = np.where(lattice > 0)
    down = np.where(lattice < 0)
    plt.scatter(*up, c='red')
    plt.scatter(*down, c='blue')
    plt.gca().set_aspect(1)
    plt.xticks([]); plt.yticks([])
    plt.show()

In [ ]:
def energy(lattice):
    inds = np.arange(N**2)
    rows = inds//N
    cols = inds%N
    subLat1 = lattice[rows,cols]
    subLat2 = lattice[rows,(cols+1)%N]
    subLat3 = lattice[(rows+1)%N,cols]
    return -np.sum(subLat1*subLat2 + subLat1*subLat3)

In [ ]:
def magnetisation(lattice):
    return np.sum(lattice)

In [ ]:
def mcstep(lattice):
    inds = rng.integers(N**2, size=(N**2))
    rands= rng.random(N**2)
    ii, jj = inds//N, inds%N
    for rand, i, j in zip(rands, ii, jj):
        site = lattice[i, j]
        deltaE = site*(lattice[i-1, j] + lattice[i, (j+1)%N] + lattice[(i+1)%N, j] + lattice[i, j-1])
        p = pacc[deltaE//2 + 2]
        if p >= 1.0:
            lattice[i, j] *= -1
        elif rand < p:
            lattice[i, j] *= -1
    return lattice

In [ ]:
# a second mcstep function for the just in time compilation
def mcstep2(lattice, inds, rands, pacc):
    ii, jj = inds//N, inds%N
    for rand, i, j in zip(rands, ii, jj):
        site = lattice[i, j]
        deltaE = site*(lattice[i-1, j] + lattice[i, (j+1)%N] + lattice[(i+1)%N, j] + lattice[i, j-1])
        p = pacc[deltaE//2 + 2]
        if p >= 1.0:
            lattice[i, j] *= -1
        elif rand < p:
            lattice[i, j] *= -1
    return lattice

In [ ]:
def statistics(ene, mag, temp):
    eneMean = np.mean(ene)
    ene2Mean= np.mean(ene**2)
    eneStd  = np.std(ene)
    magMean = np.mean(mag)
    mag2Mean= np.mean(mag**2)
    magStd  = np.std(mag)
    
    cvMean  = 1/N**2/temp*(ene2Mean - eneMean**2)
    chiMean = 1/N**2/temp*(mag2Mean - magMean**2)
    return np.array([eneMean, eneStd, magMean, magStd, cvMean, chiMean])

In [ ]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result = result[result.size // 2:]
    return result/result[0]

In [ ]:
N = 50
MC = 10000
equil = 30000
lattice = 2*rng.integers(2,size=(N,N)) - 1
temps = np.linspace(2.5,2.0,11,endpoint=True)
data  = np.empty((6,len(temps)))
ene = np.empty((MC))
mag = np.empty((MC))
eneCorr = np.empty((MC, len(temps)))
magCorr = np.empty((MC, len(temps)))

# just in time compilation and first execution
jitted_mcstep = njit()(mcstep2)
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
pacc = np.array([np.exp(-dE/3.0) for dE in range(-8,9,4)])
lattice = jitted_mcstep(lattice, inds, rands, pacc)

pbar = tqdm(total=(MC+equil)*len(temps))
for i in range(len(temps)):
    pacc = np.array([np.exp(-dE/temps[i]) for dE in range(-8,9,4)])

    pbar.set_description("T = {t:.2f}".format(t=temps[i]))

    # equilibration
    for _ in range(equil):
        inds = rng.integers(N**2, size=(N**2))
        rands= rng.random(N**2)
        lattice = jitted_mcstep(lattice, inds, rands, pacc)
        #lattice = mcstep(lattice)
        pbar.update()

    # sampling
    for step in range(MC):
        inds = rng.integers(N**2, size=(N**2))
        rands= rng.random(N**2)
        lattice = jitted_mcstep(lattice, inds, rands, pacc)
        #lattice = mcstep(lattice)
        ene[step], mag[step] = energy(lattice), magnetisation(lattice)
        pbar.update()

    data[:,i] = statistics(ene, mag, temps[i])

    eneCorr[:,i] = autocorr(ene)
    magCorr[:,i] = autocorr(mag)
    
pbar.close()

In [ ]:
def plotData(temps,data):
    plt.figure()
    plt.subplot(221)
    plt.errorbar(temps, data[0,:]/N**2,data[1,:]/N**2)
    plt.ylim(-2,0)
    plt.title('Energy')
    plt.ylabel('Energy')
    plt.xlabel('Temperature')
    plt.subplot(222)
    plt.errorbar(temps, np.abs(data[2,:])/N**2,data[3,:]/N**2)
    plt.ylim(0,1)
    plt.title('Magnetization')
    plt.ylabel('Magnetization')
    plt.xlabel('Temperature')
    plt.subplot(223)
    plt.scatter(temps, data[4,:])
    plt.ylim(0,)
    plt.title('Heat Capacity')
    plt.ylabel('Heat Capacity')
    plt.xlabel('Temperature')
    plt.subplot(224)
    plt.scatter(temps, data[5,:])
    plt.ylim(0,)
    plt.title('Magnetic Susceptibility')
    plt.ylabel('Magnetic Susceptibility')
    plt.xlabel('Temperature')
    plt.tight_layout()
    plt.show

In [ ]:
plotData(temps,data)

In [ ]:
for i in range(len(temps)):
    plt.title('Autocorr. magnetisation')
    plt.plot(magCorr[:,i], label='T=%.2f'%temps[i])
    plt.legend()
plt.show()

In [ ]:
for i in range(len(temps)):
    plt.title('Autocorr. energy')
    plt.plot(eneCorr[:,i], label='T=%.2f'%temps[i])
    plt.legend()
plt.show()